In [1]:
import torch
print(torch.__version__)

1.12.0


- Attention Block
- MultiHeadAttention
- Encoder Layer
- Pytorch Official Implementation

In [2]:
# device = torch.device('mps')
# device = torch.device('cuda')
device = torch.device('cpu')

In [3]:
X = torch.Tensor(torch.randn(8,3)).to(device)

In [4]:
# Transformer, is, the, best, model, for, representation, learning
print(X)

tensor([[-1.2556, -1.5982, -0.7095],
        [ 0.5615,  0.0871,  1.3593],
        [ 1.3082,  0.7720, -0.8341],
        [ 0.7822,  1.4809,  0.9008],
        [ 0.0059, -0.3192, -0.5622],
        [-0.4831,  0.0129, -0.0142],
        [ 1.2045,  1.3478, -0.5212],
        [-1.5952,  0.9523,  0.6453]])


In [5]:
# 학습 가능해야 하므로 parameter로 만들어준다
W_Q = torch.nn.Parameter(torch.Tensor(torch.randn(3,2))).to(device)
W_K = torch.nn.Parameter(torch.Tensor(torch.randn(3,2))).to(device)
W_V = torch.nn.Parameter(torch.Tensor(torch.randn(3,2))).to(device) 

In [7]:
# Matrix Multiplication
Q = torch.matmul(X, W_Q) # X.matmul(W_Q)
K = torch.mm(X, W_K)
V = X@W_V # broadcast 연산

In [8]:
# attention score : query와 key의 일치도 or 관련도 = Q @ K의 Transpose
# self attention이므로 Q와 K가 동일하다. 따라서 attention_score의 size는 (8,8)
attention_score = Q.matmul(K.T)

In [9]:
d_k = K.shape[1] # feature dimension = 2
print(d_k)

2


In [10]:
# scaling
attention_score = attention_score/(d_k**0.5)
# attention_score = attention_score/(math.sqrt(d_k))
# attention_score = attention_score/(torch.sqrt(d_k)) # d_k를 tensor로 변환한 후에 torch 연산할 수 있다

In [11]:
# -> 방향으로 0~1 사이의 확률값으로 변환
attention_score = torch.softmax(attention_score, dim=1) # 함수로 구현
# torch.nn.Softmax(attention_score, dim=1) # layer로 구현

In [12]:
attention_score[0,:].sum()

tensor(1., grad_fn=<SumBackward0>)

In [13]:
attention_score.sum(dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)

In [14]:
# 아래처럼 query에 d_k를 나눠주면 16번의 연산만 하면 됨(<-> 위에서는 64번의 연산 발생)
# attention score = (8,8)의 weight matrix
d_k = K.shape[1]
attention_score = (Q/(d_k**0.5)).matmul(K.T)

In [15]:
# weigted sum을 통해 (8,2) vector 생성
# selt-attention block 완성
Z = attention_score@V
print(Z.shape)

torch.Size([8, 2])
